# **Lab1: Regression**
In *lab 1*, you need to finish:

1.  Basic Part: Implement the regression model to predict people's grip force from their weight.
You can use either Matrix Inversion or Gradient Descent.


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implementing a regression model to predict grip force in a different way (for example, with more variables) than the basic part




---
# 1. Basic Part (50%)
In the first part, you need to implement the regression to predict grip force

Please save the prediction result in a CSV file and submit it to Kaggle

### Import Packages

> Note: You **cannot** import any other package


In [25]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

### Global attributes
Define the global attributes\
You can also add your own global attributes here

In [26]:
training_dataroot = 'lab1_basic_training.csv' # Training data file file named as 'lab1_basic_training.csv'
testing_dataroot = 'lab1_basic_testing.csv'   # Testing data file named as 'lab1_basic_testing.csv'
output_dataroot = 'lab1_basic.csv' # Output file will be named as 'lab1_basic.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 100 elements

### Load the Input File
First, load the basic input file **lab1_basic_training.csv** and **lab1_basic_testing.csv**

Input data would be stored in *training_datalist* and *testing_datalist*

In [27]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

### Implement the Regression Model

> Note: It is recommended to use the functions we defined, you can also define your own functions

#### Step 1: Split Data
Split data in *training_datalist* into training dataset and validation dataset


In [28]:
def SplitData(data, split_ratio):
    """
    Splits the given dataset into training and validation sets based on the specified split ratio.

    Parameters:
    - data (numpy.ndarray): The dataset to be split. It is expected to be a 2D array where each row represents a data point and each column represents a feature.
    - split_ratio (float): The ratio of the data to be used for training. For example, a value of 0.8 means 80% of the data will be used for training and the remaining 20% for validation.

    Returns:
    - training_data (numpy.ndarray): The portion of the dataset used for training.
    - validation_data (numpy.ndarray): The portion of the dataset used for validation.

    """
    training_data = []
    validation_data = []

    # TODO
    # Shuffle the data
    np.random.shuffle(data)
    
    # Calculate the index to split the data at
    split_index = int(len(data) * split_ratio)
    
    # Split the data into training and validation sets
    training_data = data[:split_index]
    validation_data = data[split_index:]

    return training_data, validation_data



#### Step 2: Preprocess Data
Handle unreasonable data and missing data

> Hint 1: Outliers and missing data can be addressed by either removing them or replacing them using statistical methods (e.g., the mean of all data).

> Hint 2: Missing data are represented as `np.nan`, so functions like `np.isnan()` can be used to detect them.

> Hint 3: Methods such as the Interquartile Range (IQR) can help detect outliers

In [29]:
def PreprocessData(data):
    """
    Preprocess the given dataset and return the result.

    Parameters:
    - data (numpy.ndarray): The dataset to preprocess. It is expected to be a 2D array where each row represents a data point and each column represents a feature.

    Returns:
    - preprocessedData (numpy.ndarray): Preprocessed data.
    """

    # TODO
    # Handle missing values (optional, assuming numerical data)    
    data = data[~np.isnan(data).any(axis=1)]
    
    # Truncate outliers based on the 1.5 IQR rule
    Q1 = np.percentile(data, 25, axis=0)
    Q3 = np.percentile(data, 75, axis=0)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Clip the values to be within the bounds
    valid_rows = np.all((data >= lower_bound) & (data <= upper_bound), axis=1)
    return data[valid_rows]


### Step 3: Implement Regression
You have to use Gradient Descent to finish this part

In [30]:
def Regression(dataset, degree):
    """
    Performs regression on the given dataset and return the coefficients.

    Parameters:
    - dataset (numpy.ndarray): A 2D array where each row represents a data point.

    Returns:
    - w (numpy.ndarray): The coefficients of the regression model. For example, y = w[0] + w[1] * x + w[2] * x^2 + ...
    """

    X = dataset[:, :-1]
    y = dataset[:, -1:]

    # Add polynomial features to X
    phi = np.ones((X.shape[0], 1))  # Add intercept term (column of ones)
    for d in range(1, degree + 1):
        phi = np.hstack((phi, (X[:, d-1:d] ** d)))  # Add x^d terms to feature matrix

    # Initialize coefficients (weights) to zero
    num_dimensions = phi.shape[1]  # Number of features (including intercept and polynomial terms)
    w = np.zeros(num_dimensions).reshape(-1, 1)

    # TODO: Set hyperparameters
    num_iteration = 100000
    learning_rate = 0.00001

    # Gradient Descent
    m = y.shape[0]
    for iteration in range(num_iteration):
        # TODO: Prediction using current weights and compute error
        y_pred = phi @ w
        error = y - y_pred

        # TODO: Compute gradient
        gradient = (-2 / m) * (phi.T @ error)

        # TODO: Update the weights
        w = w - learning_rate * gradient

        # TODO: Optionally, print the cost every 100 iterations
        if iteration % 10000 == 0:
            cost = (1 / (2 * m)) * np.sum(error ** 2)  # Mean squared error
            print(f"Iteration {iteration}, Cost: {cost}")

    return w


### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*

In [31]:

def MakePrediction(w, test_dataset):
    """
    Predicts the output for a given test dataset using a regression model.

    Parameters:
    - w (numpy.ndarray): The coefficients of the model, where each element corresponds to
                               a coefficient for the respective power of the independent variable.
    - test_dataset (numpy.ndarray): A 1D array containing the input values (independent variable)
                                          for which predictions are to be made.

    Returns:
    - list/numpy.ndarray: A list or 1d array of predicted values corresponding to each input value in the test dataset.
    """
    X = test_dataset[:, :1]
    degree = len(w) - 1
    phi = np.ones((test_dataset.shape[0], degree))  # Add intercept term (column of ones)
    
    for d in range(1, degree + 1):
        phi = np.hstack((phi, X ** d))  # Add x^d terms to feature matrix
    
    # Compute the predictions using the model coefficients (weights)
    prediction = phi @ w

    return prediction


### Step 5: Train Model and Generate Result

Use the above functions to train your model on training dataset, and predict the answer of testing dataset.

Save your predicted values in `output_datalist`

> Notice: **Remember to inclue the coefficients of your model in the report**



In [32]:
# TODO

# (1) Split data
training_data, validation_data = SplitData(training_datalist, 0.8)

# (2) Preprocess data
training_data = PreprocessData(training_data)
validation_data = PreprocessData(validation_data)

# (3) Train regression model
w = Regression(training_data, 1)

# (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth
prediction = MakePrediction(w, validation_data)
def CalculateMAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
MAPE = CalculateMAPE(validation_data[:, 1], prediction)
print('MAPE:', MAPE)

# (5) Make prediction of testing dataset and store the values in output_datalist
output_datalist = MakePrediction(w, testing_datalist) # Your prediction, should be a list with 100 elements

Iteration 0, Cost: 1055.4308961663603
Iteration 100, Cost: 40.52208377563001
Iteration 200, Cost: 40.52112279115445
Iteration 300, Cost: 40.520162939021795
Iteration 400, Cost: 40.51920421789779
Iteration 500, Cost: 40.518246626449766
Iteration 600, Cost: 40.51729016334659
Iteration 700, Cost: 40.51633482725871
Iteration 800, Cost: 40.515380616858174
Iteration 900, Cost: 40.51442753081855
MAPE: 32.40099815104526


### *Write the Output File*

Write the prediction to output csv and upload the file to Kaggle
> Format: 'Id', 'gripForce'


In [33]:
# Assume that output_datalist is a list (or 1d array) with length = 100

with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Id', 'gripForce'])
  for i in range(len(output_datalist)):
    writer.writerow([i,*output_datalist[i]])


# 2. Advanced Part (45%)
In the second part, you need to implement regression differently from the basic part to improve your grip force predictions. You must use more than two features.

You can choose either matrix inversion or gradient descent for this part

We have provided `lab1_advanced_training.csv` for your training

> Notice: Be cautious of the "gender" attribute, as it is represented by "F"/"M" rather than a numerical value.

Please save the prediction result in a CSV file and submit it to Kaggle

In [34]:
training_dataroot = 'lab1_advanced_training.csv' # Training data file file named as 'lab1_advanced_training.csv'
testing_dataroot = 'lab1_advanced_testing.csv'   # Testing data file named as 'lab1_advanced_testing.csv'
output_dataroot = 'lab1_advanced.csv' # Output file will be named as 'lab1_advanced.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 3000 elements

In [35]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  df = pd.read_csv(training_dataroot)
  df['gender'] = df['gender'].map({'M': 1, 'F': 0})
  training_datalist = df.to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  df = pd.read_csv(testing_dataroot)
  df['gender'] = df['gender'].map({'M': 1, 'F': 0})
  testing_datalist = df.to_numpy()

In [36]:
# (1) Split data
# (2) Preprocess data
# (3) Train regression model
# (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth
# (5) Make prediction of testing dataset and store the values in output_datalist

def debug(data):
    max_values = np.max(data, axis=0)
    min_values = np.min(data, axis=0)
    print("Column-wise max values:", *max_values)
    print("Column-wise min values:", *min_values)

def SplitData(data, split_ratio):
    np.random.shuffle(data)
    split_index = int(len(data) * split_ratio)
    training_data, validation_data = data[:split_index], data[split_index:]
    return training_data, validation_data

def Truncate(data, scale):
    Q1 = np.percentile(data, 25, axis=0)
    Q3 = np.percentile(data, 75, axis=0)
    IQR = Q3 - Q1
    lower_bound = Q1 - scale * IQR
    upper_bound = Q3 + scale * IQR
    valid_rows = np.all((data >= lower_bound) & (data <= upper_bound), axis=1)
    return data[valid_rows]

def PreprocessData2(data, sex):
    data = data[~np.isnan(data).any(axis=1)]
    return data

def Regression2(dataset, degree, reg_lambda=1e-4):    
    X = dataset[:, :-1]
    y = dataset[:, -1:]
    phi = np.ones((X.shape[0], 1))
    for d in range(1, degree + 1):
        phi = np.hstack((phi, (X[:, d-1:d] ** d)))
    # Compute the coefficients using Ridge Regression (with regularization)
    # w = (phi^T * phi + lambda * I)^(-1) * phi^T * y
    phi_T_phi = phi.T @ phi
    regularization_term = reg_lambda * np.eye(phi_T_phi.shape[0])  # lambda * I
    w = np.linalg.inv(phi_T_phi + regularization_term) @ phi.T @ y
    return w

def Regression3(dataset, degree):
    X = dataset[:, :-1]
    y = dataset[:, -1:]
    phi = np.ones((X.shape[0], 1))
    for d in range(1, degree + 1):
        phi = np.hstack((phi, (X[:, d-1:d] ** d)))
    w = np.zeros(phi.shape[1]).reshape(-1, 1) # Number of features (including intercept and polynomial terms)
    num_iteration = 100000
    learning_rate = 1e-35
    cost_list = []
    # Gradient Descent
    m = y.shape[0]
    for iteration in range(num_iteration):
        y_pred = phi @ w
        error = y - y_pred
        gradient = (-2 / m) * (phi.T @ error)
        w = w - learning_rate * gradient
        cost = (1 / (2 * m)) * np.sum(error ** 2)  # Mean squared error
        cost_list.append(cost)
        if iteration % 10000 == 0:
            print(f"Iteration {iteration}, Cost: {cost}")
    return w


def MakePrediction2(w, test_dataset):
    X = test_dataset
    degree = len(w) - 1
    phi = np.ones((X.shape[0], 1))
    for d in range(1, degree + 1):
        phi = np.hstack((phi, (X[:, d-1:d] ** d)))
    prediction = phi @ w
    return prediction

def calculate_mape(y_true, y_pred, epsilon=0):
    y_true_safe = np.where(y_true == 0, epsilon, y_true)
    return np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

training_data, validation_data = SplitData(training_datalist, 0.8)
male_training_data, female_training_data = PreprocessData2(training_data[training_data[:, 1] == 1], 1), PreprocessData2(training_data[training_data[:, 1] == 0], 0)
male_validation_data, female_validation_data = PreprocessData2(validation_data[validation_data[:, 1] == 1], 1), PreprocessData2(validation_data[validation_data[:, 1] == 0], 0)
print('before:', len(male_training_data), len(female_training_data), len(male_validation_data), len(female_validation_data))
for i in range(100):
    if len(male_training_data[male_training_data[:, 0] == i]) != 0:
        current_data = male_training_data[male_training_data[:, 0] == i]
        male_training_data = male_training_data[male_training_data[:, 0] != i]
        current_data = Truncate(current_data, 0.75)
        male_training_data = np.vstack((male_training_data, current_data))
    if len(male_validation_data[male_validation_data[:, 0] == i]) != 0:
        current_data = male_validation_data[male_validation_data[:, 0] == i]
        male_validation_data = male_validation_data[male_validation_data[:, 0] != i]
        current_data = Truncate(current_data, 0.75)
        male_validation_data = np.vstack((male_validation_data, current_data))
    if len(female_training_data[female_training_data[:, 0] == i]) != 0:
        current_data = female_training_data[female_training_data[:, 0] == i]
        female_training_data = female_training_data[female_training_data[:, 0] != i]
        current_data = Truncate(current_data, 0.75)
        female_training_data = np.vstack((female_training_data, current_data))
    if len(female_validation_data[female_validation_data[:, 0] == i]) != 0:
        current_data = female_validation_data[female_validation_data[:, 0] == i]
        female_validation_data = female_validation_data[female_validation_data[:, 0] != i]
        current_data = Truncate(current_data, 0.75)
        female_validation_data = np.vstack((female_validation_data, current_data))
print('after:', len(male_training_data), len(female_training_data), len(male_validation_data), len(female_validation_data))
np.random.shuffle(male_training_data)
np.random.shuffle(male_validation_data)
np.random.shuffle(female_training_data)
np.random.shuffle(female_validation_data)
w_male, w_female = Regression2(male_training_data, 7), Regression2(female_training_data, 7)
male_prediction, female_prediction = MakePrediction2(w_male, male_validation_data[:, :-1]), MakePrediction2(w_female, female_validation_data[:, :-1])
male_MAPE, female_MAPE = calculate_mape(male_validation_data[:, -1:], male_prediction), calculate_mape(female_validation_data[:, -1:], female_prediction)
print('male_MAPE:', male_MAPE)
print('female_MAPE:', female_MAPE)
# ----------
male_test_data, female_test_data = testing_datalist[testing_datalist[:, 1] == 1], testing_datalist[testing_datalist[:, 1] == 0]
male_prediction, female_prediction = MakePrediction2(w_male, male_test_data), MakePrediction2(w_female, female_test_data)
output_datalist = []
male_id, female_id = 0, 0
for sex in testing_datalist[:, 1]:
    if sex == 1: 
        output_datalist.append(male_prediction[male_id])
        male_id += 1
    else:
        output_datalist.append(female_prediction[female_id])
        female_id += 1
# output_datalist = MakePrediction2(w, testing_datalist)

with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Id', 'gripForce'])
  for i in range(len(output_datalist)):
    writer.writerow([i,*output_datalist[i]])

before: 4739 2853 1217 682
after: 2804 1556 638 329
male_MAPE: 9.720005547461898
female_MAPE: 10.489167394496437


# Save the Code File
Please save your code and submit it as an ipynb file! (**Lab1.ipynb**)